In [1]:
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

from sklearn.naive_bayes import GaussianNB


In [2]:
#LEITURA DE DADOS
treino = pd.read_csv('C:/Users/BIG DATA/Desktop/Kaggle/credit_train_label.csv')
teste = pd.read_csv('C:/Users/BIG DATA/Desktop/Kaggle/credit_test_features.csv')

In [ ]:
treino.shape

In [ ]:
teste.shape

In [ ]:
print(treino.isnull().sum())
print("===================================================")
print(teste.isnull().sum())

#A Better Option: Imputation
from sklearn.preprocessing import Imputer
from sklearn.pipeline import make_pipeline

my_imputer = Imputer()
data_with_imputed_values = my_imputer.fit_transform(original_data)
#The default behavior fills in the mean value for imputation. Statisticians have researched more complex strategies, but those complex strategies typically give no benefit once you plug the results into sophisticated machine learning models.

In [ ]:
treino.describe()

In [ ]:
teste.describe()

In [ ]:
treino.head()

In [ ]:
treino['SeriousDlqin2yrs'].sum()

In [5]:
treino['MonthlyIncome'].fillna(treino['MonthlyIncome'].mean(),inplace=True)
teste['MonthlyIncome'].fillna(treino['MonthlyIncome'].mean(),inplace=True)

In [6]:
treino['NumberOfDependents'].fillna(treino['NumberOfDependents'].mean(),inplace=True)
teste['NumberOfDependents'].fillna(treino['NumberOfDependents'].mean(),inplace=True)

In [ ]:
treino.head(n=100)

In [7]:
#PREPARO DOS DADOS
novo_treino=treino
novo_teste=teste

In [ ]:
treino.head()

In [ ]:
novo_treino.shape

In [ ]:
novo_teste.shape

In [ ]:
#Verifica se o evento é raro
sum(novo_treino['SeriousDlqin2yrs'])/len(novo_treino)

In [8]:
X=novo_treino[list(set(novo_treino.columns)-set(['SeriousDlqin2yrs','X']))]
y=novo_treino['SeriousDlqin2yrs']

In [9]:
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
seed = 42
kfold = KFold(n_splits=100, random_state=seed)
RegLog = LogisticRegression()
RegLog.fit(X_train,y_train)
results = cross_val_score(RegLog, X_train, y_train, cv=kfold)

In [13]:
from sklearn.metrics import roc_auc_score
novo_treino2 = novo_treino.drop('X',axis=1).drop('SeriousDlqin2yrs',axis=1)
auc_ = roc_auc_score(y, RegLog.predict_proba(novo_treino2)[:,1])
print("AUC: %.4f" % auc_)

AUC: 0.6275


In [15]:
submeter = pd.DataFrame()
submeter['X'] = novo_teste['X']
novo_teste2 = novo_teste.drop('X',axis=1)
submeter['SeriousDlqin2yrs'] = RegLog.predict_proba(novo_teste2)[:,1]
submeter.to_csv('submeter_Reg_Log.csv',index=False)

Classificacao

In [ ]:
cv = ShuffleSplit(n_splits=3, test_size=0.5, random_state=0)

In [16]:
# Arvore de decisão
arvore_c1 = tree.DecisionTreeClassifier()
arvore_c1.fit(X_train,y_train)
#print("C1: ",arvore_c1.score(X_test,y_test))
#scores = cross_val_score(arvore_c1, X, y, cv=cv)
#scores

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [18]:
from sklearn.metrics import roc_auc_score
novo_treino2 = novo_treino.drop('X',axis=1).drop('SeriousDlqin2yrs',axis=1)
auc_ = roc_auc_score(y, arvore_c1.predict_proba(novo_treino2)[:,1])
print("AUC: %.4f" % auc_)

AUC: 0.5639


In [40]:
#Random Forest
model_c1= RandomForestClassifier(n_estimators=500,n_jobs=5, bootstrap=True,)
model_c1.fit(X_train, y_train)
#scores = cross_val_score(model_c1, X, y, cv=cv)
#scores

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=5,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [41]:
from sklearn.metrics import roc_auc_score
novo_treino2 = novo_treino.drop('X',axis=1).drop('SeriousDlqin2yrs',axis=1)
auc_ = roc_auc_score(y, model_c1.predict_proba(novo_treino2)[:,1])
print("AUC: %.4f" % auc_)

AUC: 0.6682


In [23]:
submeter = pd.DataFrame()
submeter['X'] = novo_teste['X']
novo_teste2 = novo_teste.drop('X',axis=1)
submeter['SeriousDlqin2yrs'] = model_c1.predict_proba(novo_teste2)[:,1]
submeter.to_csv('submeter_RandomF.csv',index=False)

In [ ]:
#Gradient Boosting
grad_c1 = GradientBoostingClassifier(n_estimators=100, criterion ='mae')
grad_c1.fit(X_train, y_train)
#scores = cross_val_score(model_c1, X, y, cv=cv)
#scores

In [ ]:
from sklearn.metrics import roc_auc_score
novo_treino2 = novo_treino.drop('X',axis=1).drop('SeriousDlqin2yrs',axis=1)
auc_ = roc_auc_score(y, grad_c1.predict_proba(novo_treino2)[:,1])
print("AUC: %.4f" % auc_)

In [ ]:
#GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train_c1, y_train_c1).predict(X_train_c1)

Regressao

In [ ]:
# Arvore de decisão
arvore_c1 = tree.DecisionTreeRegressor(max_depth =3)
arvore_c1.fit(X,y)

print("C1: ",arvore_c1.score(X,y))
scores = cross_val_score(arvore_c1, X, y, cv=cv)
scores

In [ ]:
# SVM
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 100]}
svc_c1 = SVC()
clf_c1 = GridSearchCV(svc_c1, parameters)
clf_c1.fit(X_train_c1, y_train_c1)
print("C1: ",clf_c1.score(X_test_c1,y_test_c1))
scores = cross_val_score(clf_c1, X, y, cv=cv)
scores

In [ ]:
from sklearn import linear_model
LogReg1 = linear_model.LogisticRegression()
LogReg1.fit(X_train_c1, y_train_c1)

#y_pred1 = LogReg1.predict(X_test_c1)
#y_pred2 = LogReg2.predict(X_test_c2)
#y_pred3 = LogReg3.predict(X_test_c3)

#from sklearn.metrics import classification_report
#print(classification_report(y_test_c1, y_pred1))
#print(classification_report(y_test_c2, y_pred2))
#print(classification_report(y_test_c3, y_pred3))

print("C1: ",LogReg1.score(X_test_c1, y_test_c1))

In [ ]:
cv = ShuffleSplit(n_splits=3, test_size=0.5, random_state=0)

scores_1 = cross_val_score(clf_c1, X_c1, y_c1, cv=cv)
scores_2 = cross_val_score(arvore_c1, X_c1, y_c1, cv=cv)
scores_3 = cross_val_score(model_c1, X_c1, y_c1, cv=cv)
#scores_4 = cross_val_score(grad_c1, X_c1, y_c1, cv=cv)
scores_5 = cross_val_score(LogReg1, X_c1, y_c1, cv=cv)

scores1_1=scores_1.sum()
scores2_1=scores_2.sum()
scores3_1=scores_3.sum()
#scores4_1=scores_4.sum()
scores5_1=scores_5.sum()

scores1_m1=scores_1.min()
scores2_m1=scores_2.min()
scores3_m1=scores_3.min()
#scores4_m1=scores_4.min()
scores5_m1=scores_5.min()

In [ ]:
from sklearn.metrics import roc_auc_score
auc_ = roc_auc_score(y_test, probs[:,1])
print("AUC: %.4f" % auc_)

In [ ]:
print("SCORE C1 CLF = ",scores1_1,' Minimo = ',  scores1_m1)
print("SCORE C1 arvore = ",scores2_1,' Minimo = ',  scores2_m1)
print("SCORE C1 model = ",scores3_1,' Minimo = ',  scores3_m1)
print("SCORE C1 grad = ",scores4_1,' Minimo = ', scores4_m1)

In [ ]:
#prepara para a saida
submeter_c1 = pd.DataFrame()
submeter_c2 = pd.DataFrame()
submeter_c3 = pd.DataFrame()

submeter_c1['PassengerId']=teste_c1['PassengerId']
teste_c12=teste_c1.drop('PassengerId',axis=1)
submeter_c1['Survived'] = model_c1.predict(teste_c12)

submeter_1=submeter_c1.append(submeter_c2)
submeter=submeter_1.append(submeter_c3)
submeter.to_csv('submeter_5.csv',index=False)

In [ ]:
#submeter2 = submeter.sort('PassengerId')
#submeter2 = submeter.sort(['PassengerId'], ascending=[1])

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(novo_treino)
distances, indices = nbrs.kneighbors(X)
indices  